## Project 3

- **Dataset(s) to be used:** [Penn World Table version 11.0](https://www.rug.nl/ggdc/productivity/pwt/?lang=en)
- **Analysis question:** Does an increase in human capital lead to higher labor productivity? How does this relationship differ between developed and developing countries?
- **Columns that will be used:**
  - country
  - year
  - rgdpo: Output-side real GDP at chained PPPs (in mil. 2021US$)
  - emp: Number of persons engaged (in millions)
  - hc: Human capital index

- **Hypothesis**: There exists a positive relationship between labor productivity and human capital. Moreover, on average, each additional unit of human capital is associated with a larger increase in labor productivity in developed countries than in developing countries.

### Step 0. Preparation

In [81]:
import pandas as pd
import numpy as np
import plotly.express as px
import statsmodels.api as sm
from IPython.display import HTML

- **Code Interpretation:**

    - The following code performs basic data preparation using the PWT 11.0 dataset.  
    - First, it reads the third sheet of the Excel file into a pandas DataFrame.  
    - Next, it filters the dataset to keep only observations where employment (`emp`) and human capital (`hc`) are not missing.  
    - Finally, it selects a subset of relevant variables: country, year, real GDP (`rgdpo`), employment (`emp`), and human capital index (`hc`).  


In [82]:
df = pd.read_excel("pwt110.xlsx", sheet_name=2)
df = df[df["hc"].notna()]
df = df[df["emp"].notna()]
df = df[["country", "year", "rgdpo", "emp", "hc"]]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8831 entries, 94 to 13689
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   country  8831 non-null   object 
 1   year     8831 non-null   int64  
 2   rgdpo    8831 non-null   float64
 3   emp      8831 non-null   float64
 4   hc       8831 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 414.0+ KB


- **Code Interpretation:**

    - This following code constructs a new variable measuring labor productivity.  
    - It first computes the natural logarithm of real output per worker (`rgdpo / emp`) and stores it as `labor_prod`.  
    - Both the newly created productivity measure and the human capital index (`hc`) are then rounded to three decimal places to improve readability and consistency.  


In [83]:
df["labor_prod"] = np.log(df["rgdpo"] / df["emp"])
df["labor_prod"] = df["labor_prod"].round(3)
df["hc"] = df["hc"].round(3)
df.head()

,country,year,rgdpo,emp,hc,labor_prod
94,Angola,1970,32243.955078,3.197069,1.016,9.219
95,Angola,1971,34408.660156,3.257524,1.018,9.265
96,Angola,1972,34054.386719,3.341565,1.021,9.229
97,Angola,1973,36747.878906,3.441436,1.023,9.276
98,Angola,1974,36926.511719,3.545886,1.026,9.251


### Step 1. The Overall Trend

- **Code Interpretation:**
 
    - The following code generates a scatter plot of human capital (`hc`) on the x-axis and labor productivity (`labor_prod`) on the y-axis.  
    - An OLS trendline is added to summarize the relationship, and its color is manually set to red for clearer visual emphasis. 


In [84]:
fig1 = px.scatter(
    df,
    x="hc",
    y="labor_prod",
    title="Labor Productivity vs Human Capital",
    trendline="ols"
)

for trace in fig1.data:
    if trace.mode == "lines": 
        trace.line.color = "red"

HTML(fig1.to_html(include_plotlyjs="cdn", full_html=False))

- **Output Interpretation:**

    - This figure above examines the overall trend, incorporating data from all years and all countries.  
    - The scatter plot displays the relationship between human capital (`hc`) and labor productivity (`labor_prod`), where each point represents a country-year observation.  
    - The red fitted line summarizes the positive linear association between the two variables.  
    - The upward slope of the line indicates that higher human capital is generally associated with higher labor productivity across the global sample.


    

- **Code Interpretation:**

    - This following code runs a simple OLS regression to quantify the relationship between human capital and labor productivity.  
    - It specifies labor productivity (`labor_prod`) as the dependent variable and human capital (`hc`) as the explanatory variable.  
    - A constant term is added to the model, and the regression is estimated using `statsmodels`.  
    - Finally, `model.summary()` displays the full set of regression statistics, including coefficient estimates, standard errors, t-values, and goodness-of-fit measures.


In [85]:
X = df["hc"]
y = df["labor_prod"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             labor_prod   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.525
Method:                 Least Squares   F-statistic:                     9761.
Date:                Thu, 04 Dec 2025   Prob (F-statistic):               0.00
Time:                        23:41:39   Log-Likelihood:                -10544.
No. Observations:                8831   AIC:                         2.109e+04
Df Residuals:                    8829   BIC:                         2.111e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.4818      0.026    283.083      0.0

- **Output Interpretation:**

    - This table reports the results from an OLS regression of labor productivity on human capital.  
    - The estimated coefficient on human capital is positive and statistically significant: On average, a one-unit increase in the human capital index is associated with an increase of approximately 1.14 in log labor productivity, holding the intercept constant.  
    - The R-squared value of 0.525 indicates that roughly 52.5 percent of the variation in labor productivity across all observations is explained by differences in human capital alone, suggesting a strong overall relationship between the two variables.


### Step 3. The Definitions of Developed and Developing Countries

- **Code Interpretation:**

    - The following  code classifies countries based on their average labor productivity.  
    - First, it computes the mean labor productivity for each country using a `groupby` operation.  
    - It then identifies the 75th and 25th percentiles of the distribution to define cutoff values.  
    - Countries in the top quartile are labeled “Developed,” those in the bottom quartile are labeled “Developing,” and the remaining countries are assigned to the “Middle” category. 


In [86]:
country_avg = (
    df.groupby("country")["labor_prod"]
      .mean()
      .reset_index(name="avg_labor_prod")
)

q75 = country_avg["avg_labor_prod"].quantile(0.75)
q25 = country_avg["avg_labor_prod"].quantile(0.25)

country_avg["group"] = (
    country_avg["avg_labor_prod"].apply(
        lambda x: "Developed" if x >= q75 
        else ("Developing" if x <= q25 else "Middle")
    )
)

country_avg.head()


,country,avg_labor_prod,group
0,Albania,9.755185,Middle
1,Algeria,10.993422,Developed
2,Angola,9.166278,Developing
3,Argentina,9.979000,Middle
4,Armenia,9.869735,Middle


- **Code Interpretation:**

    - The following code merges the country classification back into the main dataset using a `merge` operation. 
    - By joining the original DataFrame with the table containing each country's assigned group, it adds a new variable indicating whether the country is classified as Developed, Middle, or Developing.  


In [87]:
df = df.merge(country_avg[["country", "group"]], on="country", how="left")
df.head()

,country,year,rgdpo,emp,hc,labor_prod,group
0,Angola,1970,32243.955078,3.197069,1.016,9.219,Developing
1,Angola,1971,34408.660156,3.257524,1.018,9.265,Developing
2,Angola,1972,34054.386719,3.341565,1.021,9.229,Developing
3,Angola,1973,36747.878906,3.441436,1.023,9.276,Developing
4,Angola,1974,36926.511719,3.545886,1.026,9.251,Developing


### Step 4. The Trend Among Developed Economies

- **Code Interpretation:**

    - As before, we plot a scatter diagram with human capital on the x-axis and labor productivity on the y-axis. And a fitted OLS regression line is added to summarize the relationship within this group. 
    - The difference here is that the figure uses only the subsample of countries classified as Developed.  


In [88]:
df_dev = df[df["group"] == "Developed"]

fig2 = px.scatter(
    df_dev,
    x="hc",
    y="labor_prod",
    title="Labor Productivity vs Human Capital (Developed Countries)",
    trendline="ols"
)

for trace in fig2.data:
    if trace.mode == "lines": 
        trace.line.color = "red"

HTML(fig2.to_html(include_plotlyjs="cdn", full_html=False))

In [89]:
X = df_dev["hc"]
y = df_dev["labor_prod"]
X = sm.add_constant(X)
model_dev = sm.OLS(y, X).fit()
print(model_dev.summary())

                            OLS Regression Results                            
Dep. Variable:             labor_prod   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     158.9
Date:                Thu, 04 Dec 2025   Prob (F-statistic):           2.69e-35
Time:                        23:41:39   Log-Likelihood:                -2143.6
No. Observations:                2348   AIC:                             4291.
Df Residuals:                    2346   BIC:                             4303.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.4149      0.060    174.396      0.0

- **Output Interpretation:**

    - This figure and regression table describe the relationship between human capital and labor productivity within the subsample of developed countries.  
    - The scatter plot shows a positive association, and the red fitted line summarizes the positive linear association between the two variables.  
    - The regression coefficient on human capital is 0.265, indicating that on average a one-unit increase in the human capital index is associated with an increase of about 0.27 in log labor productivity.  
    - The R-squared is 0.063, meaning that human capital explains only about 6 percent of the variation in productivity among developed economies. 

### Step 5. The Trend Among Developping Economies

- **Code Interpretation:**

    - As before, we plot a scatter diagram with human capital on the x-axis and labor productivity on the y-axis. And a fitted OLS regression line is added to summarize the relationship within this group. 
    - The difference here is that the figure uses only the subsample of countries classified as Developing.  


In [90]:
df_devp = df[df["group"] == "Developing"]

fig3 = px.scatter(
    df_devp,
    x="hc",
    y="labor_prod",
    title="Labor Productivity vs Human Capital (Developing Countries)",
    trendline="ols"
)

for trace in fig3.data:
    if trace.mode == "lines":
        trace.line.color = "red"

HTML(fig3.to_html(include_plotlyjs="cdn", full_html=False))

In [91]:
X = df_devp["hc"]
y = df_devp["labor_prod"]
X = sm.add_constant(X)
model_devp = sm.OLS(y, X).fit()
print(model_devp.summary())

                            OLS Regression Results                            
Dep. Variable:             labor_prod   R-squared:                       0.270
Model:                            OLS   Adj. R-squared:                  0.270
Method:                 Least Squares   F-statistic:                     810.8
Date:                Thu, 04 Dec 2025   Prob (F-statistic):          5.11e-152
Time:                        23:41:39   Log-Likelihood:                -1774.7
No. Observations:                2194   AIC:                             3553.
Df Residuals:                    2192   BIC:                             3565.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.2829      0.045    160.530      0.0

- **Output Interpretation:**

    - This figure and regression table summarize the relationship between human capital and labor productivity among developing countries.  
    - The scatter plot shows a clear upward pattern, and the fitted regression line indicates a strong linear positive association between variables.  
    - The estimated coefficient on human capital is 0.84, meaning that on average a one-unit increase in the human capital index is associated with an increase of roughly 0.84 in log labor productivity.  
    - The R-squared value is 0.27, implying that human capital explains about 27 percent of the cross-country and over-time variation in productivity within developing economies. 


### Step 6. Conclusion

- Across the full sample of countries and years, the analysis shows a clear positive association between human capital and labor productivity. The baseline regression indicates a substantial relationship, with a coefficient of approximately 1.14 and an R-squared of about 0.53, suggesting that higher levels of human capital are strongly correlated with higher productivity overall.

- When separating countries by development level, the patterns diverge. Among developed economies, the relationship becomes much weaker: the coefficient declines to about 0.27 and the R-squared falls to roughly 0.06, indicating limited explanatory power. In contrast, developing countries exhibit a much stronger association. The coefficient rises to about 0.84 and the R-squared improves to roughly 0.27, implying that improvements in human capital are more strongly linked to productivity gains in lower-income settings. These results suggest that developing economies may benefit more from policies that expand and strengthen human capital, such as investments in higher education, skills training, and workforce development.

- Several limitations should be noted. The analysis relies solely on simple linear regressions, which do not account for omitted variable bias, reverse causality, or unobserved heterogeneity; therefore, the results cannot be interpreted as causal effects. Moreover, the relatively modest R-squared values across models indicate that human capital alone explains only part of the variation in productivity. Future research could incorporate richer specifications, additional controls, non-linear models, or causal identification strategies to obtain more robust insights.
